In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
# !pip install scikit-learn keras tensorflow
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.neural_network import MLPClassifier
from keras.models import Sequential
from keras.layers import Dense, LSTM, Conv2D, Flatten
import numpy as np


In [4]:
file_path_credit = 'creditcard_data_featured.csv'
creditcard_data = pd.read_csv(file_path_credit)

# Feature and Target Separation
# For creditcard dataset
X_creditcard = creditcard_data.drop('Class', axis=1)  # Features
y_creditcard = creditcard_data['Class']               # Target

# Train-Test Split (80% train, 20% test by default)
X_creditcard_train, X_creditcard_test, y_creditcard_train, y_creditcard_test = train_test_split(
    X_creditcard, y_creditcard, test_size=0.2, random_state=42)

# Print shapes of the resulting datasets
print("Creditcard Train:", X_creditcard_train.shape, y_creditcard_train.shape)
print("Creditcard Test:", X_creditcard_test.shape, y_creditcard_test.shape)

Creditcard Train: (226980, 30) (226980,)
Creditcard Test: (56746, 30) (56746,)


In [5]:
def evaluate_model(model, X_train, y_train, X_test, y_test):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred)

    return accuracy, report

In [6]:
# Initialize a dictionary to store model performances
model_performance = {}

# 1. Logistic Regression
log_model = LogisticRegression(max_iter=1000)
accuracy, report = evaluate_model(log_model, X_creditcard_train, y_creditcard_train, X_creditcard_test, y_creditcard_test)
model_performance['Logistic Regression'] = (accuracy, report)

In [7]:
# 2. Decision Tree
dt_model = DecisionTreeClassifier()
accuracy, report = evaluate_model(dt_model, X_creditcard_train, y_creditcard_train, X_creditcard_test, y_creditcard_test)
model_performance['Decision Tree'] = (accuracy, report)

In [8]:
# 3. Random Forest
rf_model = RandomForestClassifier()
accuracy, report = evaluate_model(rf_model, X_creditcard_train, y_creditcard_train, X_creditcard_test, y_creditcard_test)
model_performance['Random Forest'] = (accuracy, report)

In [9]:
# 4. Gradient Boosting
gb_model = GradientBoostingClassifier()
accuracy, report = evaluate_model(gb_model, X_creditcard_train, y_creditcard_train, X_creditcard_test, y_creditcard_test)
model_performance['Gradient Boosting'] = (accuracy, report)

In [10]:
# 5. Multi-Layer Perceptron (MLP)
mlp_model = MLPClassifier(max_iter=1000)
accuracy, report = evaluate_model(mlp_model, X_creditcard_train, y_creditcard_train, X_creditcard_test, y_creditcard_test)
model_performance['Multi-Layer Perceptron'] = (accuracy, report)

In [11]:
# 6. Convolutional Neural Network (CNN)


from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Flatten, Dense
# Reshape the data for CNN (assuming a single feature input)
X_creditcard_train_cnn = X_creditcard_train.values.reshape(X_creditcard_train.shape[0], X_creditcard_train.shape[1], 1)
X_creditcard_test_cnn = X_creditcard_test.values.reshape(X_creditcard_test.shape[0], X_creditcard_test.shape[1], 1)

# Adjust the kernel size to be smaller or equal to the input dimensions
cnn_model = Sequential([
    Conv1D(32, 3, activation='relu', input_shape=(X_creditcard_train_cnn.shape[1], X_creditcard_train_cnn.shape[2])), # Changed to Conv1D and adjusted kernel size
    Flatten(),
    Dense(1, activation='sigmoid')
])
cnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
cnn_model.fit(X_creditcard_train_cnn, y_creditcard_train, epochs=10, batch_size=32, verbose=0)


c:\Users\Hermela\Documents\Fraud\.venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [12]:
import tensorflow as tf
# Before calling evaluate, ensure data types are compatible with TensorFlow:
X_creditcard_test_cnn = X_creditcard_test_cnn.astype(np.float32)  # Convert to float32
y_creditcard_test = tf.cast(y_creditcard_test, tf.float32)  # Use tf.cast to convert to float32

# Additionally, check for and handle NaN/infinity values:
X_creditcard_test_cnn = np.nan_to_num(X_creditcard_test_cnn)  # Replace NaN and infinity with finite values
# For TensorFlow tensors, you can use tf.where to handle NaN/infinity
y_creditcard_test = tf.where(tf.math.is_finite(y_creditcard_test), y_creditcard_test, 0.0) #If infinite or Nan, replace with 0

cnn_loss, cnn_accuracy = cnn_model.evaluate(X_creditcard_test_cnn, y_creditcard_test, verbose=0)
model_performance['Convolutional Neural Network'] = (cnn_accuracy, 'N/A')  # No report for CNN

In [13]:
# 7. Recurrent Neural Network (RNN)
# Reshape for RNN (assumes time steps of 1)
X_creditcard_train_rnn = X_creditcard_train.values.reshape(X_creditcard_train.shape[0], 1, X_creditcard_train.shape[1])
X_creditcard_test_rnn = X_creditcard_test.values.reshape(X_creditcard_test.shape[0], 1, X_creditcard_test.shape[1])

rnn_model = Sequential([
    LSTM(32, input_shape=(X_creditcard_train_rnn.shape[1], X_creditcard_train_rnn.shape[2])),
    Dense(1, activation='sigmoid')
])
rnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
rnn_model.fit(X_creditcard_train_rnn, y_creditcard_train, epochs=10, batch_size=32, verbose=0)

c:\Users\Hermela\Documents\Fraud\.venv\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [14]:
# Before evaluating the RNN, ensure data types are compatible with TensorFlow:
X_creditcard_test_rnn = X_creditcard_test_rnn.astype(np.float32)  # Convert to float32
# If y_creditcard_test was modified earlier, reset it to its original type
# Use tf.cast for TensorFlow tensors
y_creditcard_test = tf.cast(y_creditcard_test, tf.int64) if isinstance(y_creditcard_test, tf.Tensor) else y_creditcard_test.astype(np.int64)

# Convert y_creditcard_test to a TensorFlow tensor with float32 type
y_creditcard_test = tf.cast(y_creditcard_test, tf.float32)

rnn_loss, rnn_accuracy = rnn_model.evaluate(X_creditcard_test_rnn, y_creditcard_test, verbose=0)
model_performance['Recurrent Neural Network'] = (rnn_accuracy, 'N/A')  # No report for RNN

In [15]:
# 8. Long Short-Term Memory (LSTM)
# Using the same RNN model as the LSTM in this example

lstm_model = Sequential([
    LSTM(32, input_shape=(X_creditcard_train_rnn.shape[1], X_creditcard_train_rnn.shape[2])),
    Dense(1, activation='sigmoid')
])
lstm_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
lstm_model.fit(X_creditcard_train_rnn, y_creditcard_train, epochs=10, batch_size=32, verbose=0)
lstm_loss, lstm_accuracy = lstm_model.evaluate(X_creditcard_test_rnn, y_creditcard_test, verbose=0)
model_performance['Long Short-Term Memory'] = (lstm_accuracy, 'N/A')  # No report for LSTM

In [16]:
# Print model performance results
for model, (accuracy, report) in model_performance.items():
    print(f"{model} Accuracy: {accuracy:.4f}")
    if report != 'N/A':
        print(report)

Logistic Regression Accuracy: 0.9992
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56656
           1       0.88      0.54      0.67        90

    accuracy                           1.00     56746
   macro avg       0.94      0.77      0.84     56746
weighted avg       1.00      1.00      1.00     56746

Decision Tree Accuracy: 0.9990
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56656
           1       0.68      0.72      0.70        90

    accuracy                           1.00     56746
   macro avg       0.84      0.86      0.85     56746
weighted avg       1.00      1.00      1.00     56746

Random Forest Accuracy: 0.9996
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56656
           1       0.99      0.73      0.84        90

    accuracy                           1.00     56746
   macro avg       0.99      

# Model Interpretation for creditcard

## 1. Logistic Regression
- **Accuracy**: 0.9992
- **Interpretation**: This model demonstrates high accuracy overall but struggles significantly with detecting fraud (class 1). The precision for fraud detection is 0.88, which indicates that when it predicts fraud, it is correct 88% of the time. However, the recall is only 0.54, meaning it identifies just 54% of actual fraud cases. The F1-score of 0.67 shows a lack of balance between precision and recall, suggesting that the model is biased towards predicting non-fraud cases due to the class imbalance.

## 2. Decision Tree
- **Accuracy**: 0.9991
- **Interpretation**: The Decision Tree model has similar accuracy to Logistic Regression but performs slightly better in detecting fraud. It achieves a precision of 0.69 and a recall of 0.73 for fraud cases, indicating a better ability to identify actual frauds. The F1-score of 0.71 suggests that while it can capture some fraudulent transactions, there is still significant room for improvement, especially in reducing false positives.

## 3. Random Forest
- **Accuracy**: 0.9995
- **Interpretation**: The Random Forest model exhibits the highest accuracy and shows a strong capability in detecting fraud. It achieves a precision of 0.96, meaning that when it predicts fraud, it is highly reliable. The recall for fraud cases is 0.73, indicating that it correctly identifies 73% of actual frauds. The F1-score of 0.83 signifies a good balance between precision and recall, making it a robust model for fraud detection.

## 4. Gradient Boosting
- **Accuracy**: 0.9993
- **Interpretation**: Similar to Random Forest, Gradient Boosting also demonstrates high accuracy. It has a precision of 0.89 and a recall of 0.63 for fraud detection. While it is reasonably good at identifying fraud cases, the lower recall indicates it misses a larger portion of actual frauds compared to Random Forest. The F1-score of 0.74 shows it balances precision and recall but suggests room for enhancement in fraud detection capabilities.

## 5. Multi-Layer Perceptron (MLP)
- **Accuracy**: 0.9995
- **Interpretation**: The MLP model achieves similar accuracy to Random Forest and Gradient Boosting. It has a precision of 0.87 and a recall of 0.80 for fraud cases, indicating a strong performance in identifying fraud. The F1-score of 0.83 suggests that the model maintains a good balance between precision and recall, indicating it can effectively detect fraudulent transactions while managing false positives.

## 6. Long Short-Term Memory (LSTM)
- **Accuracy**: 0.9995
- **Interpretation**: The LSTM model shows high accuracy but lacks detailed metrics in the report. Generally, LSTMs are effective for sequential data and could capture temporal patterns in fraud, but further metrics are needed to evaluate its precision and recall accurately.

## 7. Convolutional Neural Network (CNN)
- **Accuracy**: 0.9994
- **Interpretation**: Similar to LSTM, the CNN shows high accuracy but does not provide detailed performance metrics. CNNs are typically used for image data but can also be adapted for structured data. Without specific precision and recall values, the model's performance in detecting fraud remains unclear.

## 8. Recurrent Neural Network (RNN)
- **Accuracy**: 0.9995
- **Interpretation**: Like the LSTM, the RNN achieves high accuracy without detailed evaluation metrics. RNNs are designed for sequential data analysis and may be useful in identifying patterns over time, but again, further metrics are needed to fully interpret its effectiveness in fraud detection.

# Summary
Overall, the Random Forest model stands out for its high precision and reasonable recall, making it the most effective model for fraud detection among those evaluated. Logistic Regression, while achieving high accuracy, shows significant weaknesses in identifying fraud cases due to class imbalance. Further improvements can be made across all models, particularly in recall for fraud detection, to enhance their effectiveness.


# Model Explainability

In [19]:
# Import necessary libraries
import numpy as np
import shap
import pandas as pd
from lime.lime_tabular import LimeTabularExplainer
import matplotlib.pyplot as plt
from scipy.special import expit as sigmoid

In [20]:
def shap_explainability(model, X_train, X_test):
    """
    Generates SHAP explanations for a given model and data.

    Parameters:
        model: The trained model to explain.
        X_train: The training data used to train the model.
        X_test: The test data to explain.

    Returns:
        None. Displays SHAP plots.
    """

    # Convert input data to numeric types
    X_train = X_train.astype(float)
    X_test = X_test.astype(float)

    # Create a SHAP explainer
    masker = shap.maskers.Independent(X_train)
    shap_explainer = shap.Explainer(model.predict, masker)
    
    # Get SHAP values for the test data
    shap_values = shap_explainer(X_test)
    
    # SHAP summary plot
    shap.plots.beeswarm(shap_values[:, :, 0])
    
    # SHAP force plot
    shap.plots.force(shap_values[0, :, 0])
    
    # SHAP dependence plot
    shap.plots.scatter(shap_values[:, :, 0])

In [21]:
def shap_explainability_neural(model, X_train, X_test):

    # Convert input data to numeric types
    X_train = X_train.astype(float)
    X_test = X_test.astype(float)
      

    # Create a SHAP explainer
    masker = shap.maskers.Independent(X_train)
    model_proba = lambda x: sigmoid(model.predict(x))
    shap_explainer = shap.Explainer(model_proba, masker, algorithm='permutation', nsamples=100, verbose=0)
    
    # Get SHAP values for the test data
    shap_values = shap_explainer(X_test)
    
    # SHAP summary plot
    shap.plots.beeswarm(shap_values)
    
    # SHAP force plot
    shap.plots.force(shap_values[0,:])
    
    # SHAP dependence plot
    shap.plots.scatter(shap_values[:, "feature_name"],  ylabel="SHAP value\n(higher means more likely to fraud)")

In [22]:
def lime_explainability(model, X_train, X_test):
    # Convert X_train to a pandas DataFrame
    X_train_df = pd.DataFrame(X_train)
    
    # Create a LIME explainer
    lime_explainer = LimeTabularExplainer(X_train_df.values, feature_names=X_train_df.columns, class_names=["Not Fraud", "Fraud"], discretize_continuous=True)
    
    # Get LIME explanations for the test data
    lime_explanation = lime_explainer.explain_instance(X_test.iloc[0].values, model.predict, num_features=5)    
    # LIME feature importance plot
    lime_explanation.as_pyplot_figure()

## Logistic Regression

In [23]:
# Explain the evaluated models
print("Explaining Logistic Regression model with SHAP...")
shap_explainability(log_model, X_creditcard_train, X_creditcard_test)

Explaining Logistic Regression model with SHAP...


PermutationExplainer explainer: 56747it [23:31, 39.97it/s]                              


IndexError: too many indices for array

In [ ]:
print("Explaining Logistic Regression model with LIME...")
lime_explainability(log_model, X_fraud_train, X_fraud_test)

## Decision Tree

In [ ]:
# Explain the evaluated models
print("Explaining Decision Tree model with SHAP...")
shap_explainability(dt_model, X_fraud_train, X_fraud_test)

In [ ]:
print("Explaining Decision Tree model with LIME...")
lime_explainability(dt_model, X_fraud_train, X_fraud_test)

## Random Forest

In [ ]:
# Explain the evaluated models
print("Explaining Random Forest model with SHAP...")
shap_explainability(rf_model, X_fraud_train, X_fraud_test)

In [ ]:
print("Explaining Random Forest model with LIME...")
lime_explainability(rf_model, X_fraud_train, X_fraud_test)

## Gradient Boosting

In [ ]:
# Explain the evaluated models
print("Explaining Gradient Boosting model with SHAP...")
shap_explainability(gb_model, X_fraud_train, X_fraud_test)

In [ ]:
print("Explaining Gradient Boosting model with LIME...")
lime_explainability(gb_model, X_fraud_train, X_fraud_test)

## Multiple Layer Perceptron

In [ ]:
# Explain the evaluated models
print("Explaining Multi-Layer Perceptron (MLP) model with SHAP...")
shap_explainability(mlp_model, X_fraud_train, X_fraud_test)

In [ ]:
print("Explaining Multi-Layer Perceptron (MLP) model with LIME...")
lime_explainability(mlp_model, X_fraud_train, X_fraud_test)

## Convolutional Neural Networks

In [ ]:
# Explain the evaluated models
print("Explaining Convolutional Neural Network (CNN) model with SHAP...")
shap_explainability_neural(cnn_model, X_fraud_train, X_fraud_test)

In [ ]:
print("Explaining Convolutional Neural Network (CNN) model with LIME...")
lime_explainability(cnn_model, X_fraud_train, X_fraud_test)

## Recurrent Neural Network

In [ ]:
import shap

# Convert reshaped data to float32
X_fraud_train_reshaped = X_fraud_train_reshaped.astype('float32')
X_fraud_test_reshaped = X_fraud_test_reshaped.astype('float32')

# Use GradientExplainer or KernelExplainer for compatibility
shap_explainer = shap.GradientExplainer(rnn_model, X_fraud_train_reshaped)

# Get SHAP values
shap_values = shap_explainer.shap_values(X_fraud_test_reshaped)

# SHAP summary plot
shap.summary_plot(shap_values, X_fraud_test_reshaped, feature_names=X_fraud_train.columns)

In [ ]:
print("Explaining Recurrent Neural Network (RNN) model with LIME...")
lime_explainability(rnn_model, X_fraud_train, X_fraud_test)

## Saving my models

In [24]:
import os
import pickle
import shap

# Directory to store models and SHAP visualizations
output_dir = "models-creditcard"
os.makedirs(output_dir, exist_ok=True)

# Models dictionary
models = {
    'dt_model': dt_model,
    'rf_model': rf_model,
    'gb_model': gb_model,
    'log_model': log_model,
    'mlp_model': mlp_model,
    'cnn_model': cnn_model,
    'rnn_model': rnn_model,
    'lstm_model': lstm_model
}


In [18]:
# Save each model as a .pkl file in the directory
for model_name, model in models.items():
    with open(os.path.join(output_dir, f"{model_name}.pkl"), 'wb') as model_file:
        pickle.dump(model, model_file)
        print(f"Saved {model_name} to {output_dir}/{model_name}.pkl")

Saved dt_model to models-creditcard/dt_model.pkl
Saved rf_model to models-creditcard/rf_model.pkl
Saved gb_model to models-creditcard/gb_model.pkl
Saved log_model to models-creditcard/log_model.pkl
Saved mlp_model to models-creditcard/mlp_model.pkl
Saved cnn_model to models-creditcard/cnn_model.pkl
Saved rnn_model to models-creditcard/rnn_model.pkl
Saved lstm_model to models-creditcard/lstm_model.pkl
